In [1]:
# the data formatting might be a problem
import csv
import math
import os  
import time 

from keras._tf_keras.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras._tf_keras.keras.metrics import MeanAbsoluteError, Accuracy, Precision, Recall, MeanSquaredError
from keras._tf_keras.keras.models import Sequential
from keras._tf_keras.keras.optimizers import Adam , RMSprop, Nadam
from keras._tf_keras.keras.layers import LSTM, Dense, Dropout, Bidirectional, BatchNormalization, Masking, InputLayer, Conv1D, MaxPooling1D, Flatten, TimeDistributed, LayerNormalization, Activation
from keras._tf_keras.keras.regularizers import L1L2, L1, L2
from keras._tf_keras.keras.utils import to_categorical
import numpy as np 
import pandas as pd

from sklearn.calibration import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import StandardScaler
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder 

from FeatureEngineering import FeatureEngineering as fe 

In [2]:
def create_dataframe_from_data(input_path: str):
    dataframes = []
    for file_name in os.listdir(input_path):
        file_path = os.path.join(input_path, file_name)

        data = np.load(file_path, allow_pickle=True)
        df = pd.DataFrame(data)

        gesture = file_name.split("_")[0]
        gesture_index = int(file_name.split("_")[1].split(".")[0])
        
        # add two new columns to df 
        df["gesture"] = gesture
        df["gesture_index"] = gesture_index

        dataframes.append(df)
        df.sort_values(by="frame", inplace=True)
    
    return pd.concat(dataframes, ignore_index=True) if len(dataframes) > 0 else ValueError("Dataframe is empty")

def create_dict_from_df(df: pd.DataFrame):
    diction = {x: [] for x in np.unique(df["gesture"].values.tolist())}
    for gesture_index, gesture_data in df.groupby("gesture_index"):
        gesture = np.unique(gesture_data["gesture"].values.tolist())[0]
        tmp = diction[gesture] + [gesture_index]
        diction.update({gesture:tmp})
    return diction

def transform_to_sequences(df: pd.DataFrame, sequence_length, target: str, additional_targets: list = None):
    sequences = []
    labels = []
    grouped = df.groupby('gesture_index')
    
    for _, group in grouped:
        group = group.sort_values('frame').reset_index(drop=True)
        for i in range(0, len(group), sequence_length):
            sequence = group.iloc[i:i+sequence_length].drop(columns=['gesture_index']).values # gesture_index is dropped because it's more like metadata, no value as a features
            if len(sequence) == sequence_length:
                sequences.append(sequence)
                labels.append(group.iloc[0][[target] + additional_targets]) if additional_targets else labels.append(group.iloc[0][target])
    return np.array(sequences), np.array(labels) 

def split_dataset(df: pd.DataFrame, target_label: str, sequence_length=30, additional_targets: list=None, train_ratio=0.7, val_ratio=0.15 ,test_ratio=0.15):
    # probably make this train, val, test
    assert train_ratio + val_ratio + test_ratio == 1.0, "ratios must sum to 1."

    gesture_index_dict = create_dict_from_df(df)

    train_indices = []
    val_indices = []
    test_indices = []

    for _, indices in gesture_index_dict.items():
        n_total = len(indices)
        n_train = math.ceil(n_total * train_ratio)
        n_val = math.ceil(n_total * val_ratio)
        
        train_indices.extend(indices[:n_train])
        val_indices.extend(indices[n_train:n_train + n_val])
        test_indices.extend(indices[n_train + n_val:])

    grouped_data = df.groupby("gesture_index")

    train_frames, val_frames, test_frames = [], [], []
    for idx in train_indices:
        train_frames.append(grouped_data.get_group(idx))
        
    for idx in val_indices:
        val_frames.append(grouped_data.get_group(idx))
        
    for idx in test_indices:
        test_frames.append(grouped_data.get_group(idx))

    # Concatenate the dataframes to create the final train and test sets
    train_set = pd.concat(train_frames).reset_index(drop=True)
    val_set = pd.concat(val_frames).reset_index(drop=True)
    test_set = pd.concat(test_frames).reset_index(drop=True)
    
    # X_train_sequences, y_train_sequences = transform_to_sequences(train_set,  sequence_length, target_label, additional_targets)
    # X_val_sequences, y_val_sequences = transform_to_sequences(val_set,  sequence_length, target_label, additional_targets)
    # X_test_sequences, y_test_sequences = transform_to_sequences(test_set,  sequence_length, target_label, additional_targets)

    # Separate X and y
    # X_train = X_train_sequences
    # y_train = y_train_sequences
    # X_val = X_val_sequences
    # y_val = y_val_sequences
    # X_test = X_test_sequences
    # y_test = y_test_sequences

    X_train = train_set.drop(columns=[target_label])
    y_train = train_set[[target_label] + additional_targets] if additional_targets else train_set[[target_label]]
    X_val = val_set.drop(columns=[target_label])
    y_val = val_set[[target_label] + additional_targets] if additional_targets else train_set[[target_label]]
    X_test = test_set.drop(columns=[target_label])
    y_test = test_set[[target_label] + additional_targets] if additional_targets else test_set[[target_label]]


    return X_train, y_train, X_val, y_val, X_test, y_test

def save_sequences_to_csv(sequences, labels, filename, df):
    with open(filename, mode="w", newline='') as file:
        writer = csv.writer(file)
        header = []      
        for i in range(30):
            for col in df.columns[:-2]:  # Exclude 'gesture_index' and 'gesture' columns
                header.append(f"{col}_frame_{i}")
        header.append("gesture")
        writer.writerow(header)
        
        # Write data
        for sequence, label in zip(sequences, labels):
            flattened_sequence = sequence.flatten()
            writer.writerow(np.append(flattened_sequence, label))

def calculate_hand_motion_feature(df: pd.DataFrame, landmark_cols: list):
    df_copy = df.copy()
    s = time.process_time()
    df_temporal = fe.calculate_temporal_features(df_copy, landmark_cols)
    print(time.process_time()-s)
    # Ensure there are no duplicate columns
    df_combined = df_copy.loc[:,~df_copy.columns.duplicated()]
    return df_combined

def preprocess_pipeline(timeseries_columns, numerical_columns, categorical_columns):
    ts_numerical_transformer = Pipeline(steps=[
        ('imputer', KNNImputer(n_neighbors=5)), # might want to change this out back to the interpolatioon methods
        ('imputer2', SimpleImputer(strategy="mean")),
        ('scaler', StandardScaler())
    ])

    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy="mean")),
        ("normalize", StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy="most_frequent")), # technically this is wrong
        ("ohe", OneHotEncoder(sparse_output=False))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('ts_num', ts_numerical_transformer, timeseries_columns),
            ('num', numerical_transformer, numerical_columns),
            # ('cat', categorical_transformer, categorical_columns)
        ],
        remainder='passthrough',
        sparse_threshold=0,
        n_jobs=-1
    )
 
    preprocessor.set_output(transform="pandas")
    
    return preprocessor


In [4]:

input_path = r'C:\Users\Gen3r\Documents\capstone\ml_model\data\data_3'
dataframe = create_dataframe_from_data(input_path)

X_train, y_train, X_val, y_val, X_test, y_test = split_dataset(dataframe, target_label='gesture')
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

landmark_columns = [f"{col}" for col in dataframe.columns if col.startswith(("hx", "hy", "hz", "px", "py", "pz", "lx", "ly", "lz", "rx", "ry", "rz"))]
categorical_columns = ["gesture_index"]
numerical_columns = ["frame", "frame_rate", "frame_width", "frame_height"] + [f"{col}" for col in dataframe.columns if col.startswith("pose_visibility")]
derived_features =  [f"{feat}_{col}" for feat in ["velocity", "acceleration", "jerk"] for col in landmark_columns if col.startswith(("lx", "ly", "lz", "rx", "ry", "rz"))]
time_series_columns = landmark_columns + derived_features     
res = [item for item in landmark_columns if item.startswith(("r", "l"))]


X_train_fe = calculate_hand_motion_feature(X_train, res)
X_val_fe = calculate_hand_motion_feature(X_val, res)
X_test_fe = calculate_hand_motion_feature(X_test, res)

preprocessor = preprocess_pipeline(time_series_columns, numerical_columns, categorical_columns)

X_train_transformed = preprocessor.fit_transform(X_train_fe)
X_val_transformed = preprocessor.transform(X_val_fe)
X_test_transformed = preprocessor.transform(X_test_fe)

(7320, 1667) (7320, 1) (1650, 1667) (7320, 1) (1470, 1667) (1470, 1)
53.859375
10.5625
9.53125


In [12]:
def transform_to_sequences(df: pd.DataFrame, sequence_length, target: str, additional_targets: list = None):
    sequences = []
    labels = []
    grouped = df.groupby('gesture_index')
    
    for _, group in grouped:
        group = group.sort_values('frame').reset_index(drop=True)
        for i in range(0, len(group) - sequence_length + 1, sequence_length):
            sequence = group.iloc[i:i+sequence_length].drop(columns=['gesture_index']).values
            if len(sequence) == sequence_length:
                sequences.append(sequence)
                labels.append(group.iloc[0][target]) if additional_targets is None else labels.append(group.iloc[0][[target] + additional_targets])
    return np.array(sequences), np.array(labels)

# Assuming the target column name is 'gesture' and no additional targets
def create_sequences_with_labels(X_transformed, y, sequence_length):
    # Combine features and labels into a single DataFrame
    combined_df = pd.concat([pd.DataFrame(X_transformed), y.reset_index(drop=True)], axis=1)
    
    # Convert the DataFrame to sequences
    X_sequences, y_sequences = transform_to_sequences(combined_df, sequence_length, target='gesture')
    
    return X_sequences, y_sequences


print(X_train_transformed.shape)
print(X_val_transformed.shape)
print(X_test_transformed.shape)


(7320, 2045)
(1650, 2045)
(1470, 2045)
